In [2]:
%load_ext watermark
%watermark -v -m  -u -n -p pandas,numpy,textwrap,matplotlib,sklearn,gseapy -a Filippo_Valle -g -r -b -w

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Filippo_Valle 
last updated: Tue Jun 30 2020 

CPython 3.7.6
IPython 7.15.0

pandas 1.0.4
numpy 1.18.5
textwrap unknown
matplotlib 3.2.1
sklearn 0.0
gseapy 0.9.18

compiler   : GCC 7.5.0
system     : Linux
release    : 4.19.76-linuxkit
machine    : x86_64
processor  : x86_64
CPU cores  : 2
interpreter: 64bit
Git hash   : 728c5fcb2ae65fb2d06e3b61b8b167d388b13f95
Git repo   : git@github.com:fvalle1/topics.git
Git branch : develop
watermark 2.0.2


In [ ]:
import numpy as np
import pandas as pd
from textwrap import wrap
from matplotlib import pyplot as plt
from sklearn import metrics
import matplotlib.ticker as mticker
import sys, os
from hsbmpy import plot_topic_size, get_max_available_L
from hypergeom import parameters_for_hypergeometric, build_map, plot_map

In [ ]:
directory="/home/jovyan/work/phd/datasets/cancers/breast"
os.chdir(directory)
sys.path.append('/home/jovyan/work/phd/')

In [ ]:
L = get_max_available_L(directory)

In [ ]:
df = pd.read_csv("mainTable.csv", index_col=[0])

# Geneontology

In [ ]:
from geneontology import get_ontology_df, ensg_to_symbol
from tableanalyser import get_symbol
import gseapy as gs

In [ ]:
l=L-1
print(f"level {l}")
algorithm = "topsbm"
df_topics = pd.read_csv(f"{directory}/{algorithm}/{algorithm}_level_{l}_topics.csv")
df_topics_smooth = pd.read_csv(f"{directory}/{algorithm}/{algorithm}_level_{l}_word-dist.csv",index_col=0)
df_topics_smooth.index = [g[:15] for g in df_topics_smooth.index]

In [ ]:
def get_topic_over_thr(topic_name, q=0.75):
    topic = df_topics_smooth[topic_name]
    topic = topic[topic>0]
    topic = topic[topic>topic.quantile(q=q)]
    return topic.sort_values(ascending=False)

In [ ]:
df_symbols= pd.read_csv("https://www.genenames.org/cgi-bin/download/custom?col=gd_hgnc_id&col=gd_app_sym&col=gd_pub_ensembl_id&col=md_ensembl_id&col=md_eg_id&status=Approved&status=Entry%20Withdrawn&hgnc_dbtag=on&order_by=gd_app_sym_sort&format=text&submit=submit", index_col=[0], sep='\t')

In [ ]:
def get_sea():
    for g in df_topics.values.ravel()[[str(s)!='nan' for s in df_topics.values.ravel()]]:
        yield get_symbol(g)

In [ ]:
with open("gback.txt",'w') as f:
    list(map(lambda x: f.writelines(x+'\n')if len(x)>1 else None, get_sea()))
    
with open("gback_ensg.txt",'w') as f:
    list(map(lambda x: f.writelines(x[:15]+'\n')if len(x)>1 else None, df_topics.values.ravel()[[str(s)!='nan' for s in df_topics.values.ravel()]]))

In [ ]:
gs.get_library_name()

In [ ]:
#https://www.gsea-msigdb.org/gsea/downloads.jsp
gene_sets = ['GO_Molecular_Function_2018',
             'GO_Biological_Process_2018',
             'GO_Cellular_Component_2018',
             'Human_Phenotype_Ontology',
             'WikiPathways_2019_Human',
             '/home/jovyan/work/phd/MSigDB/c1.all.v7.1.symbols.gmt',
             '/home/jovyan/work/phd/MSigDB/c2.all.v7.1.symbols.gmt',
             '/home/jovyan/work/phd/MSigDB/c3.all.v7.1.symbols.gmt',
             '/home/jovyan/work/phd/MSigDB/c4.all.v7.1.symbols.gmt',
             '/home/jovyan/work/phd/MSigDB/c5.all.v7.1.symbols.gmt',
             '/home/jovyan/work/phd/MSigDB/c6.all.v7.1.symbols.gmt',
             '/home/jovyan/work/phd/MSigDB/c7.all.v7.1.symbols.gmt',
            ]

In [ ]:
threshhold = 5e-1
cutoff = 5e-1
background = len([g for g in get_sea()])
os.system("mkdir -p gsea")
for itopic,topic in enumerate(df_topics.columns):
    try:
        enriched_topic = pd.read_csv("gsea/gsea_level_%d_topic_%d.csv"%(l,itopic+1), index_col=[0])
        print(topic)
    except:
        try:
            gene_list = ensg_to_symbol(df_topics.loc[:,topic].dropna().values)
            #gene_list = ensg_to_symbol(get_topic_over_thr(topic).index)
            print(topic)
            enriched_topic = get_ontology_df(gene_list, cutoff=cutoff, threshhold = threshhold, gene_sets = gene_sets, background=background)
            enriched_topic = enriched_topic.sort_values(by=['Adjusted P-value'], ascending=True)[:20]
            enriched_topic.to_csv("gsea/gsea_level_%d_topic_%d.csv"%(l,itopic+1))
        except:
            print(*sys.exc_info())
            continue
    print(enriched_topic)

In [ ]:
topic_pvalues = []
topic_gos = []
for itopic,topic in enumerate(df_topics.columns):
    try:
        enriched_topic = pd.read_csv("gsea/gsea_level_%d_topic_%d.csv"%(l,itopic+1))
        if len(enriched_topic.index) >0:
            p_val = np.sort(enriched_topic['Adjusted P-value'])[0]
            topic_pvalues.append(-np.log10(p_val))
            for goc in enriched_topic['Gene_set'][:10].unique():
                topic_gos.append(goc)
        print(topic)
    except:
        print("error", sys.exc_info()[0])

In [ ]:
fig = plt.figure(figsize=(18,15))
x = np.arange(1,1+len(topic_pvalues))
c, _, _ = plt.hist(topic_pvalues, histtype='step', lw=20, bins=45, color="gray")
plt.vlines(-np.log10(0.05),0,np.max(c)*1.1, color="red", ls='--', lw=10, label="$\\alpha=0.05$")
plt.xlabel('-log(P-value)', fontsize=35)
plt.ylabel("number of topics", fontsize=35)
#plt.ylim(0,0.055)
#plt.yscale('log')
plt.legend(fontsize=35)
plt.tick_params(which="both",labelsize=35)
fig.savefig("%s/pvaluescrosstopic(%d).pdf"%(directory,l))

In [ ]:
fig = plt.figure(figsize=(18,20))
gos, goscounts = np.unique(topic_gos, return_counts=True)
plt.barh(["\n".join(wrap(str(l).replace('_',' '),20)) for l in gos], goscounts)
plt.yticks(fontsize=15)
plt.tick_params(which="both",labelsize=35)
plt.tight_layout()
plt.show()
fig.savefig("%s/pvaluecategories(%d).pdf"%(directory,l))

# hypergeometric operlaps

In [ ]:
from scipy.stats import hypergeom
from sklearn.metrics import v_measure_score
import seaborn as sns
sns.set_context('paper')

In [ ]:
import importlib, hypergeom
importlib.reload(hypergeom)
from hypergeom import *

In [ ]:
hsbm_list_topics = pd.read_csv("topsbm/topsbm_level_3_topics.csv")
gene_list = hsbm_list_topics.values.ravel().astype(str)
gene_list = list(map(lambda g: g[:15],filter(lambda g: g!="nan", gene_list)))
hsbm_list=pd.Series(index=[g[:15] for g in gene_list], dtype=str)
for topic in hsbm_list_topics.columns:
    hsbm_list[[g[:15] for g in hsbm_list_topics[topic].dropna()]]=topic

In [ ]:
hsbm_list_topics = pd.read_csv("topsbm-log/topsbm-log_level_3_topics.csv")
gene_list = hsbm_list_topics.values.ravel().astype(str)
gene_list = list(map(lambda g: g[:15],filter(lambda g: g!="nan", gene_list)))
hsbm_log_list=pd.Series(index=[g[:15] for g in gene_list], dtype=str)
for topic in hsbm_list_topics.columns:
    hsbm_log_list[[g[:15] for g in hsbm_list_topics[topic].dropna()]]=topic

In [ ]:
wgcna_list_topics = pd.read_csv("wgcna/wgcna_level_0_topics.csv")
gene_list = wgcna_list_topics.values.ravel().astype(str)
gene_list = list(map(lambda g: g[:15],filter(lambda g: g!="nan", gene_list)))
wgcna_list=pd.Series(index=np.unique([g[:15] for g in gene_list]), dtype=str)
for topic in wgcna_list_topics.columns:
    wgcna_list[np.unique([g[:15] for g in wgcna_list_topics[topic].dropna()])]=topic

In [ ]:
tm_list_topics = pd.read_csv("tm/tm_level_0_topics.csv")
gene_list = tm_list_topics.values.ravel().astype(str)
gene_list = list(map(lambda g: g[:15],filter(lambda g: g!="nan", gene_list)))
tm_list=pd.Series(index=np.unique([g[:15] for g in gene_list]), dtype=str)
for topic in tm_list_topics.columns:
    tm_list[np.unique([g[:15] for g in tm_list_topics[topic].dropna()])]=topic

In [ ]:
lda_list_topics = pd.read_csv("lda/lda_level_1_topics.csv")
gene_list = lda_list_topics.values.ravel().astype(str)
gene_list = list(map(lambda g: g[:15],filter(lambda g: g!="nan", gene_list)))
lda_list=pd.Series(index=[g[:15] for g in gene_list], dtype=str)
for topic in lda_list_topics.columns:
    lda_list[lda_list.index.isin([g[:15] for g in lda_list_topics[topic].dropna()])]=topic
lda_list=lda_list.reset_index().drop_duplicates("index").set_index("index")
lda_list=pd.Series(index=lda_list.index, data=lda_list.values.ravel())

In [ ]:
def run(first_name, last_name):
    list_1 = globals()[f"{first_name}_list"]
    list_2 = globals()[f"{last_name}_list"]

    #to uniform
    list_1 = list_1[list_1.index.isin(list_2.index)]
    list_2 = list_2[list_2.index.isin(list_1.index)]
    hyper_params = parameters_for_hypergeometric(list_1, list_2)
    df_cmap = build_map(*hyper_params)
    df_cmap[df_cmap<3]=0
    df_cmap = df_cmap.sort_values(by=[t for t in df_cmap.columns], ascending=False)
    plot_map(df_cmap, first_name=first_name, last_name=last_name)

In [ ]:
run("hsbm","tm")
run("hsbm","lda")
run("hsbm","wgcna")
run("tm","lda")
run("tm","wgcna")
run("lda","wgcna")

In [ ]:
print("hsbm & tm %.3f \\\\ \hline"%v_measure_score(hsbm_list.reindex_like(tm_list), tm_list))
print("hsbm & lda %.3f \\\\ \hline"%v_measure_score(hsbm_list.reindex_like(lda_list).dropna(), lda_list.reindex_like(hsbm_list).dropna()))
print("hsbm & wgcna %.3f \\\\ \hline"%v_measure_score(hsbm_list.reindex_like(wgcna_list), wgcna_list))
print("tm & lda %.3f \\\\ \hline"%v_measure_score(tm_list.reindex_like(lda_list).dropna(), lda_list.reindex_like(tm_list).dropna()))
print("tm & wgcna %.3f \\\\ \hline"%v_measure_score(tm_list.reindex_like(wgcna_list).dropna(), wgcna_list.reindex_like(tm_list).dropna()))
print("lda & wgcna %.3f \\\\ \hline"%v_measure_score(lda_list.reindex_like(wgcna_list).dropna(), wgcna_list.reindex_like(lda_list).dropna()))

In [ ]:
for g in hsbm_list[hsbm_list=="Topic 1"].index:
    print(g)